# Différentes approches de NLP

## Données

In [1]:
import pandas as pd
df = pd.read_csv("../data/tableau.csv")

## Regex

Extraire toutes les dates 

In [6]:
#import re
#m = '(\d{1,2}\s+\w+\s+\d{4})'


## Mot par mot, ou dictionnaires

comptons certains mots 
- startup
- entreprise
- etc.

## DFM

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#modele.get_feature_names_out()

array(['00', '000', '000e', ..., 'œuvre', 'œuvrent', 'œuvres'],
      dtype=object)

Pas mal d'options

In [14]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

## TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


Peut être pas la meilleure échelle : décomposer au paragraphe ?

## Premiers pas avec SpaCy


### Télécharger et utiliser un modèle (un trained pipepeline)

Se familiariser avec la structure doc/span/token

In [2]:
#pip install spacy

Installer le pipeline

In [3]:
#!python -m spacy download fr_core_news_lg

In [ ]:
pipeline = spacy.load("fr_core_news_lg")

Possibilité d'avoir la distance entre deux textes

In [ ]:
# Compare two documents
doc1 = pipeline("J'aime beaucoup programmer en Python")
doc2 = pipeline("J'aime beaucoup programmer en R")
print(doc1.similarity(doc2))

### Lemmatiser et garder que les noms

Utiliser le POS

In [8]:
# Partie à faire en TP

### Identifier les entités nommées

In [ ]:
text = df["text"].iloc[0]
doc = pipeline(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Afficher les entités nommées.

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", 
                jupyter=True)

## GliNER

Un modèle plus complexe pour l'identification d'entités

In [32]:
#!pip install gliner